# Real time kalibratie test

Het proces van extrinsieke kalibratie dat we vorige opdracht hebben uitgetest kan ook realtime gedaan worden door gebruik te maken van je webcam. Je dient hiervoor het dambordpatroon af te printen en op een hard vlak (karton) te kleven. Als je het patroon voor de camera houdt, dan zal de extrinsieke matrix realtime berekend worden en kan je virtueel gedefiniëerde objecten in het target frame live zien verschijnen op je dambord. Dit is dus hoe Augmented Reality werkt. 

# 1. Importeer biliotheken


> Deze bibliotheken zijn nodig voor het volbrengen van dit script. In de CameraCalibration bibliotheek (die je vindt in de Classes folder op je Google Drive) vind je de code achter de commando's die je hier zal gebruiken. Deze cel zal een error geven als je niet met je Google Drive bent verbonden. Zorg er dus voor dat je links in de bestanden een folder 'drive' ziet staan naast de default 'sample_data'-folder. Indien niet, verbind met de drive door links op het mapje te klikken en vervolgens op het Google Drive symbool te klikken. Druk vervolgens nog eens op 'Runtime', 'Runtime opnieuw starten', om te refreshen. 

In [ ]:
import sys
sys.path.append('/content/drive/My Drive/object_pose_estimation_online')
from Classes.CameraCalibration import *
from google.colab.patches import cv2_imshow

# 2. Script om toegang te krijgen tot je webcam

> Dit is een script om je webcam te kunnen uitlezen. Hier hoef je niets mee te doen.



In [ ]:
#@title javascript_code
import base64
import html
import io
import time

from IPython.display import display, Javascript
from google.colab.output import eval_js
import numpy as np
from PIL import Image
import cv2

def start_input():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 512, 512);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 512; //video.videoWidth;
      captureCanvas.height = 512; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function takePhoto(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

# 4. Laden van de berekende intrinsieke camera matrix

> In vorige opgave heb je de intrinsieke matrix opgeslagen, deze werd automatisch opgeslagen onder /content/drive/My Drive/object_pose_estimation_online/data/matrix_files/intrinsic_camera_properties.npz. Deze kunnen we nu laden. De functie "load_intrinsic_camera_matrix" geeft je niet enkel de intrinsieke matrix, maar ook de distrotiecoëfficiënten nodig voor de kalibratie. Merk op dat een intrinsieke matrix eigen is aan de camera. De vorig berekende matrix geldt dus enkel voor de camera in het labo. Hoe dan ook zal je een goed effect krijgen met deze matrix.

In [ ]:
# Load intrinsic camera matrix and distortion coefficients
intrinsic_camera_matrix_file = "/content/drive/My Drive/object_pose_estimation_online/data/matrix_files/intrinsic_camera_properties.npz"
mtx, dist = load_intrinsic_camera_matrix(intrinsic_camera_matrix_file)

# 4. Toon realtime resultaten


> We definiëren twee zaken:


*   De coordinaten van het target assenstelsel. We definiëren de oorsprong op (X, Y, Z) = (0, 0, 0) in het camera frame, de x-as op (X, Y, Z) = (0.06, 0, 0) , de y-as op (X, Y, Z) = (0, 0.06, 0), en de z-as op (X, Y, Z) = (0, 0, 0.06). 
*   De coördinaten van een virtuele kubus van 0.04x0.04x0.04 meter in het target frame.


> Voor deze beide zaken maken we gebruik van de intrinsieke en extrinsieke matrix om te kijken met welke pixels deze overeenkomen. Als we deze pixels berekenen en projecteren op het beeld, dan krijgen we het effect alsof deze zaken zich werkelijk in de ruimte (gedefiniëerd in target frame) bevinden.

In [ ]:
# Axis coordinates in world coordinates system
axis = np.float32([[0.06, 0, 0], [0, 0.06, 0], [0, 0, 0.06], [0, 0, 0]])
box = np.float32([[0.08, 0.06, 0], [0.12, 0.06, 0], [0.12, 0.10, 0], [0.08, 0.10, 0], [0.08, 0.06, -0.04],
                          [0.12, 0.06, -0.04], [0.12, 0.10, -0.04], [0.08, 0.10, -0.04]]).reshape(-1, 3)



> Als je dit script runt zal je onderaan je webcam output zien verschijnen. Let op dat er niets gebeurt als er geen dambordpatroon wordt gedetecteerd. De processing kan wat traag verlopen zodat ze in het begin wat dingen ziet die niet correct zijn. Hou hiervoor je dambordpatroon wat plat (niet frontaal tov de webcam) en hou dit stabiel. Je kan het filmen stoppen door op 'Runtime => Uitvoering Onderbreken' te klikken of door je webtabblad te sluiten. 



In [ ]:
# Start webcam
start_input()
label_html = 'Capturing...'
img_data = ''

# Loop
while True:

    # Get image from webcam
    data = eval_js('takePhoto("{}", "{}")'.format(label_html, img_data))
    jpeg_bytes = base64.b64decode(data['img'].split(',')[1])
    image_PIL = Image.open(io.BytesIO(jpeg_bytes))
    image_array = np.array(image_PIL)

    # Turn image to grayscale
    gray = image_to_grayscale(image_array)  

    # Find the corners in the chessbord calibration tool
    corners = find_corners(gray)

    # Creating empty drawing image
    drawing_array = np.zeros([512,512,4], dtype=np.uint8)  

    # Get extrinsic camera calibration matrix (target - camera transform)
    if not corners is None:

      # Prepare object points in 3D space in meters
      objp = get_object_points()

      # Extrinsic camera calibration
      ct_transform = extrinsic_calibration(objp, corners, mtx, dist)

      # Retrieve rotation matrix and translation vector from ct_transform
      ct_rot = ct_transform[:3, :3]
      ct_trans = ct_transform[0:3, 3] 
    
      # Project 3D axis points to pixel coordinates
      imgpts, jac = cv2.projectPoints(axis, ct_rot, ct_trans, mtx, dist)

      # Draw axis on image
      drawing_array = draw_axis(drawing_array, imgpts.astype(int))

      # Project 3D box points to pixel coordinates
      imgpts, jac = cv2.projectPoints(box, ct_rot, ct_trans, mtx, dist)

      # Draw box on image
      drawing_array = draw_box1(drawing_array, imgpts.astype(int))
      drawing_array[:,:,3] = (drawing_array.max(axis = 2) > 0 ).astype(int) * 255

    # Send new image to javascript
    drawing_PIL = Image.fromarray(drawing_array, 'RGBA')
    iobuf = io.BytesIO()
    drawing_PIL.save(iobuf, format='png')
    drawing_bytes = 'data:image/png;base64,{}'.format((str(base64.b64encode(iobuf.getvalue()), 'utf-8')))
    img_data = drawing_bytes
    